# Importing Libraries & Data

In [1]:
import pandas as pd
import numpy as np
from google.colab import files
eps = np.finfo(float).eps
from numpy import log2 as log

uploaded = files.upload()

Saving t.csv to t.csv


# Showing Data

In [2]:
df = pd.read_csv('t.csv')
print(df)

   Gender  Car Ownership Travel Cost Income Level Transportation Mode
0    Male              0       Cheap          Low                 Bus
1    Male              1       Cheap       Medium                 Bus
2  Female              1       Cheap       Medium               Train
3  Female              0       Cheap          Low                 Bus
4    Male              1       Cheap       Medium                 Bus
5    Male              0    Standard       Medium               Train
6  Female              1    Standard       Medium               Train
7  Female              1   Expensive         High                 Car
8    Male              2   Expensive       Medium                 Car
9  Female              2   Expensive         High                 Car


# Shape of Data

In [3]:
np.shape(df)

(10, 5)

# Entropy Calculation & Decision Tree Building

In [6]:
def find_entropy(df):
    Class = df.keys()[-1]   
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
    return entropy
  
  
def find_entropy_attribute(df, attribute):
  Class = df.keys()[-1]  
  target_variables = df[Class].unique()  #This gives all 'Yes' and 'No'
  variables = df[attribute].unique()    #This gives different features in that attribute (like 'Cheap','Expensive' in Travel Cost)
  entropy2 = 0
  for variable in variables:
      entropy = 0
      for target_variable in target_variables:
          num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
          den = len(df[attribute][df[attribute]==variable])
          fraction = num/(den+eps)
          entropy += -fraction*log(fraction+eps)
      fraction2 = den/len(df)
      entropy2 += -fraction2*entropy
  return abs(entropy2)


def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(IG)]
  
  
def get_subtable(df, node,value):
  return df[df[node] == value].reset_index(drop=True)


def buildTree(df,tree=None): 
    Class = df.keys()[-1]   
    #Here we build our decision tree Get attribute with maximum information gain
    node = find_winner(df)
    #Get distinct value of that attribute e.g Travel Cost is node and 'Cheap','Expensive' and 'Standard' are values
    attValue = np.unique(df[node])
    #Create an empty dictionary to create tree    
    if tree is None:                    
        tree={}
        tree[node] = {}
   #We make loop to construct a tree by calling this function recursively. 
    #In this we check if the subset is pure and stops if it is pure. 
    for value in attValue:
        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable['Transportation Mode'],return_counts=True)                        
        if len(counts)==1:#Checking purity of subset
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable) #Calling the function recursively 
                   
    return tree

# Showing the Tree

In [7]:
tree = buildTree(df)
import pprint
pprint.pprint(tree)

{'Travel Cost': {'Cheap': {'Gender': {'Female': {'Car Ownership': {0: 'Bus',
                                                                   1: 'Train'}},
                                      'Male': 'Bus'}},
                 'Expensive': 'Car',
                 'Standard': 'Train'}}


# Doing Predictions

In [9]:
def predict(train,tree):
    '''
    Function to predict for any input variable.
    '''
    #Recursively we go through the tree that we built earlier

    for nodes in tree.keys():        
        # print('nodes=',nodes)
        value = train[nodes]
        # print('value=',value)
        tree = tree[nodes][value]
        # print('tree=',tree)
        prediction = 0
            
        if type(tree) is dict:
            # print('type(tree) is dict=',type(tree), dict)
            prediction = predict(train, tree)
            # print('if=', prediction)
        else:
            prediction = tree
            # print('else=', prediction)
            break;                            
        # print()
    return prediction

In [10]:
data = [{'Gender':'Male', 'Car Ownership':1, 'Travel Cost':'Standard', 'Income Level':'High'}, {'Gender':'Male', 'Car Ownership':0, 'Travel Cost':'Cheap', 'Income Level':'Medium'}, 
         {'Gender':'Female', 'Car Ownership':1, 'Travel Cost':'Cheap', 'Income Level':'High'}]
inst = np.array(data)
i = 0
for el in data: 
  #print(el)
  prediction = predict(el, tree)
  print(prediction)
  i += 1

Train
Bus
Train
